In [46]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [47]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))

샘플의 수 : 11314


In [48]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [49]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [50]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

<ipython-input-50-b4124dfb5e6a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [51]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [52]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.

In [54]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


TF-IDF 행렬 만들기

In [55]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [56]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [57]:
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (11314, 1000)


토픽 모델링

In [58]:
svd_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

10

In [59]:
np.shape(svd_model.components_)

NameError: ignored

In [60]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('like', 0.21386), ('know', 0.20046), ('people', 0.19293), ('think', 0.17805), ('good', 0.15128)]
Topic 2: [('thanks', 0.32888), ('windows', 0.29088), ('card', 0.18069), ('drive', 0.17455), ('mail', 0.15111)]
Topic 3: [('game', 0.37064), ('team', 0.32443), ('year', 0.28154), ('games', 0.2537), ('season', 0.18419)]
Topic 4: [('drive', 0.53324), ('scsi', 0.20165), ('hard', 0.15628), ('disk', 0.15578), ('card', 0.13994)]
Topic 5: [('windows', 0.40399), ('file', 0.25436), ('window', 0.18044), ('files', 0.16078), ('program', 0.13894)]
Topic 6: [('chip', 0.16114), ('government', 0.16009), ('mail', 0.15625), ('space', 0.1507), ('information', 0.13562)]
Topic 7: [('like', 0.67086), ('bike', 0.14236), ('chip', 0.11169), ('know', 0.11139), ('sounds', 0.10371)]
Topic 8: [('card', 0.46633), ('video', 0.22137), ('sale', 0.21266), ('monitor', 0.15463), ('offer', 0.14643)]
Topic 9: [('know', 0.46047), ('card', 0.33605), ('chip', 0.17558), ('government', 0.1522), ('video', 0.14356)]
Topic 10

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


LDA실행 코드 

In [61]:
!pip install genism

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement genism (from versions: none)
ERROR: No matching distribution found for genism


In [62]:
from gensim import corpora 
dictionary = corpora.Dictionary(tokenized_doc) 
corpus = [dictionary.doc2bow(text) for text in tokenized_doc] 
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [63]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.010*"available" + 0.008*"anonymous" + 0.006*"information" + 0.006*"contact"')
(1, '0.006*"braves" + 0.005*"weaver" + 0.005*"cursor" + 0.005*"netcom"')
(2, '0.012*"ground" + 0.010*"engine" + 0.010*"wire" + 0.007*"wiring"')
(3, '0.010*"armenian" + 0.009*"israel" + 0.009*"jews" + 0.009*"armenians"')
(4, '0.012*"like" + 0.012*"know" + 0.010*"would" + 0.009*"time"')
(5, '0.006*"hawks" + 0.004*"adams" + 0.003*"apostles" + 0.003*"nords"')
(6, '0.036*"jesus" + 0.019*"bible" + 0.014*"christ" + 0.011*"church"')
(7, '0.024*"file" + 0.018*"output" + 0.017*"entry" + 0.013*"program"')
(8, '0.013*"would" + 0.011*"people" + 0.008*"think" + 0.006*"many"')
(9, '0.017*"char" + 0.017*"scsi" + 0.007*"byte" + 0.005*"column"')
(10, '0.026*"space" + 0.007*"nasa" + 0.006*"data" + 0.005*"printf"')
(11, '0.011*"drive" + 0.009*"would" + 0.008*"like" + 0.008*"card"')
(12, '0.020*"mail" + 0.016*"list" + 0.014*"please" + 0.014*"send"')
(13, '0.013*"year" + 0.012*"game" + 0.008*"last" + 0.007*"good"')
(14, '0.

In [64]:
print(ldamodel.print_topics())

[(0, '0.010*"available" + 0.008*"anonymous" + 0.006*"information" + 0.006*"contact" + 0.006*"center" + 0.005*"data" + 0.005*"computer" + 0.005*"university" + 0.004*"includes" + 0.004*"volume"'), (1, '0.006*"braves" + 0.005*"weaver" + 0.005*"cursor" + 0.005*"netcom" + 0.005*"colorado" + 0.005*"pitcher" + 0.004*"swap" + 0.004*"houston" + 0.004*"larson" + 0.003*"reds"'), (2, '0.012*"ground" + 0.010*"engine" + 0.010*"wire" + 0.007*"wiring" + 0.007*"picture" + 0.006*"neutral" + 0.006*"plastic" + 0.004*"sleeve" + 0.004*"electrical" + 0.004*"circuits"'), (3, '0.010*"armenian" + 0.009*"israel" + 0.009*"jews" + 0.009*"armenians" + 0.009*"people" + 0.007*"turkish" + 0.006*"said" + 0.006*"israeli" + 0.005*"turkey" + 0.004*"jewish"'), (4, '0.012*"like" + 0.012*"know" + 0.010*"would" + 0.009*"time" + 0.007*"back" + 0.007*"think" + 0.007*"said" + 0.007*"something" + 0.007*"going" + 0.006*"could"'), (5, '0.006*"hawks" + 0.004*"adams" + 0.003*"apostles" + 0.003*"nords" + 0.003*"smythe" + 0.003*"wales"

문서별 토픽 분포 보기 

In [65]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(3, 0.2154464), (8, 0.34465182), (19, 0.42619213)]
1 번째 문서의 topic 비율은 [(3, 0.027881302), (4, 0.25536373), (8, 0.5322827), (9, 0.13961843), (14, 0.026561089)]
2 번째 문서의 topic 비율은 [(1, 0.0491879), (3, 0.35602933), (4, 0.14973652), (8, 0.39269793), (11, 0.04025154)]
3 번째 문서의 topic 비율은 [(1, 0.20776817), (4, 0.1799992), (11, 0.069908984), (12, 0.016938904), (16, 0.51419073)]
4 번째 문서의 topic 비율은 [(4, 0.2801133), (17, 0.68655336)]


In [66]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [67]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,19.0,0.4262,"[(3, 0.21545184), (8, 0.34465116), (19, 0.4261..."
1,1,8.0,0.5323,"[(3, 0.027881343), (4, 0.25539014), (8, 0.5322..."
2,2,8.0,0.3927,"[(1, 0.04918807), (3, 0.35603106), (4, 0.14976..."
3,3,16.0,0.5142,"[(1, 0.20776743), (4, 0.18000905), (11, 0.0698..."
4,4,17.0,0.6864,"[(4, 0.2802799), (17, 0.68638676)]"
5,5,8.0,0.3545,"[(4, 0.13292517), (5, 0.21952607), (6, 0.19587..."
6,6,5.0,0.6461,"[(5, 0.64608824), (8, 0.04718778), (10, 0.0279..."
7,7,8.0,0.3205,"[(3, 0.14909668), (4, 0.20083946), (6, 0.08753..."
8,8,18.0,0.3074,"[(0, 0.034564245), (4, 0.18024746), (8, 0.2430..."
9,9,11.0,0.4657,"[(4, 0.29005527), (8, 0.07505698), (9, 0.10296..."
